In [40]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
data = pd.read_csv('./data/train.csv')
data.head()

,label,imdbId,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,tmdbId
0,0,1980209,25000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...",http://www.painandgainmovie.com/,"[{""id"": 416, ""name"": ""miami""}, {""id"": 11454, ""...",en,Pain & Gain,"Daniel Lugo, manager of the Sun Gym in 1990s M...",31.893932,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-04-18,86175291,130.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Their American dream is bigger than yours.,Pain & Gain,134374
1,1,101507,6500000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",NaN,"[{""id"": 542, ""name"": ""street gang""}, {""id"": 57...",en,Boyz n the Hood,Boyz n the Hood is the popular and successful ...,17.705671,"[{""name"": ""Columbia Pictures Corporation"", ""id...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1991-07-12,57504069,112.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Once upon a time in South Central L.A... It ai...,Boyz n the Hood,650
2,0,1575694,6000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",NaN,[],en,The Hit List,A disgruntled man creates a hit list with a st...,4.957014,"[{""name"": ""North by Northwest Entertainment"", ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2011-04-02,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Who's on your list?,The Hit List,58626
3,0,422295,16800000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 9648, ""na...",http://www.furmovie.com/,"[{""id"": 1003, ""name"": ""photographer""}, {""id"": ...",en,Fur: An Imaginary Portrait of Diane Arbus,In 1958 New York Diane Arbus is a housewife an...,4.358255,"[{""name"": ""River Road Entertainment"", ""id"": 12...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2006-08-30,2281089,122.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",A love story.,Fur: An Imaginary Portrait of Diane Arbus,18615
4,1,1135503,40000000,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 18, ...",http://www.julieandjulia.com/,"[{""id"": 1918, ""name"": ""cooking""}, {""id"": 3096,...",en,Julie & Julia,Julia Child and Julie Powell – both of whom wr...,22.853006,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-08-06,129540499,123.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Based on Two True Stories,Julie & Julia,24803


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1976 entries, 0 to 1975
Data columns (total 19 columns):
label                   1976 non-null int64
imdbId                  1976 non-null int64
budget                  1976 non-null int64
genres                  1976 non-null object
homepage                741 non-null object
keywords                1976 non-null object
original_language       1976 non-null object
original_title          1976 non-null object
overview                1976 non-null object
popularity              1976 non-null float64
production_companies    1976 non-null object
production_countries    1976 non-null object
release_date            1976 non-null object
revenue                 1976 non-null int64
runtime                 1976 non-null float64
spoken_languages        1976 non-null object
tagline                 1676 non-null object
title                   1976 non-null object
tmdbId                  1976 non-null int64
dtypes: float64(2), int64(5), object(12)
m

In [29]:
test = pd.read_csv('./data/test.csv')
test.head()

,imdbId,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,tmdbId
0,118615,45000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 27, ""...",NaN,"[{""id"": 5968, ""name"": ""amazon""}, {""id"": 10787,...",en,Anaconda,"A ""National Geographic"" film crew is taken hos...",27.684779,"[{""name"": ""Columbia Pictures Corporation"", ""id...","[{""iso_3166_1"": ""BR"", ""name"": ""Brazil""}, {""iso...",1997-04-11,136885767,89.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",When You Can't Breathe You Can't Scream.,Anaconda,9360
1,1990314,2500000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",http://robotandfrank-film.com/,"[{""id"": 6054, ""name"": ""friendship""}, {""id"": 14...",en,Robot & Frank,"A delightful dramatic comedy, a buddy picture,...",23.419061,"[{""name"": ""Samuel Goldwyn Films"", ""id"": 9118},...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-08-16,3325038,85.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Friendship doesn't have an off switch,Robot & Frank,84329
2,49513,4500000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...",NaN,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,Moby Dick,"In 1841, young Ishmael signs up for service ab...",5.224593,"[{""name"": ""Warner Bros."", ""id"": 6194}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1956-06-27,10400000,116.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",From the immortal adventure classic...of whali...,Moby Dick,10339
3,112851,7000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 28, ""n...",NaN,"[{""id"": 75, ""name"": ""gunslinger""}, {""id"": 139,...",en,Desperado,A gunslinger is embroiled in a war with a loca...,25.511217,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-08-25,25405445,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","When the smoke clears, it just means he's relo...",Desperado,8068
4,1077368,150000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 14, ""nam...",http://darkshadowsmovie.warnerbros.com,"[{""id"": 616, ""name"": ""witch""}, {""id"": 2883, ""n...",en,Dark Shadows,Vampire Barnabas Collins is inadvertently free...,50.306728,"[{""name"": ""Village Roadshow Pictures"", ""id"": 7...","[{""iso_3166_1"": ""AU"", ""name"": ""Australia""}, {""...",2012-05-08,245527149,113.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Every Family Has Its Demons,Dark Shadows,62213


In [30]:
X_test = test[['budget', 'popularity', 'revenue', 'runtime']]

In [15]:
data = data.drop('imdbId', axis=1)
data = data.drop('tmdbId', axis=1)
data.head()

,label,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title
0,0,25000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...",http://www.painandgainmovie.com/,"[{""id"": 416, ""name"": ""miami""}, {""id"": 11454, ""...",en,Pain & Gain,"Daniel Lugo, manager of the Sun Gym in 1990s M...",31.893932,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-04-18,86175291,130.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Their American dream is bigger than yours.,Pain & Gain
1,1,6500000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",NaN,"[{""id"": 542, ""name"": ""street gang""}, {""id"": 57...",en,Boyz n the Hood,Boyz n the Hood is the popular and successful ...,17.705671,"[{""name"": ""Columbia Pictures Corporation"", ""id...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1991-07-12,57504069,112.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Once upon a time in South Central L.A... It ai...,Boyz n the Hood
2,0,6000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",NaN,[],en,The Hit List,A disgruntled man creates a hit list with a st...,4.957014,"[{""name"": ""North by Northwest Entertainment"", ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2011-04-02,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Who's on your list?,The Hit List
3,0,16800000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 9648, ""na...",http://www.furmovie.com/,"[{""id"": 1003, ""name"": ""photographer""}, {""id"": ...",en,Fur: An Imaginary Portrait of Diane Arbus,In 1958 New York Diane Arbus is a housewife an...,4.358255,"[{""name"": ""River Road Entertainment"", ""id"": 12...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2006-08-30,2281089,122.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",A love story.,Fur: An Imaginary Portrait of Diane Arbus
4,1,40000000,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 18, ...",http://www.julieandjulia.com/,"[{""id"": 1918, ""name"": ""cooking""}, {""id"": 3096,...",en,Julie & Julia,Julia Child and Julie Powell – both of whom wr...,22.853006,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-08-06,129540499,123.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Based on Two True Stories,Julie & Julia


In [22]:
new_data = data[['budget', 'popularity', 'revenue', 'runtime']]
new_data.head()

,budget,popularity,revenue,runtime
0,25000000,31.893932,86175291,130.0
1,6500000,17.705671,57504069,112.0
2,6000000,4.957014,0,90.0
3,16800000,4.358255,2281089,122.0
4,40000000,22.853006,129540499,123.0


In [23]:
y=data.iloc[:,:1]
y.head()

,label
0,0
1,1
2,0
3,0
4,1


In [36]:
X_train, X_test, y_train, y_test = train_test_split(new_data, y)

In [37]:
dt = tree.DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [38]:
y_pre = dt.predict(X_test)

In [39]:
dt.score(X_test, y_test)

0.6417004048582996

In [57]:
text_data = data[['overview']]
text_data.head()

,overview
0,"Daniel Lugo, manager of the Sun Gym in 1990s M..."
1,Boyz n the Hood is the popular and successful ...
2,A disgruntled man creates a hit list with a st...
3,In 1958 New York Diane Arbus is a housewife an...
4,Julia Child and Julie Powell – both of whom wr...


In [59]:
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(text_data, y)

In [41]:
bow = CountVectorizer()

In [49]:
X_bow_train = bow.fit_transform(X_train_text)

In [56]:
X_bow_train_df = pd.DataFrame(X_bow_train.toarray())
X_bow_train_df

,0
0,1


In [52]:
X_extended = pd.concat([X_train, X_bow_train], axis=1) #X_train + X_bow_train
X_extended.head()

TypeError: cannot concatenate object of type "<class 'scipy.sparse.csr.csr_matrix'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [51]:
dt_text = tree.DecisionTreeClassifier()
dt_text.fit()